In [1]:
!rm .fleche -rf

# Getting Started with Fleche

This notebook demonstrates the main features of the `fleche` library, a caching library for Python.

## Long-running calculation

In [2]:
import time
from fleche import fleche, cache, tags, project
from fleche.digest import Digest

No config file found. Using default memory cache.


In [3]:
@fleche
def long_running_calculation(x):
    print(f'Running calculation for {x}...')
    time.sleep(2)
    return x * x

In [4]:
start = time.time()
long_running_calculation(2)
end = time.time()
print(f'First call took {end - start:.2f} seconds.')

Running calculation for 2...


First call took 2.00 seconds.


In [5]:
start = time.time()
long_running_calculation(2)
end = time.time()
print(f'Second call took {end - start:.2f} seconds.')

Second call took 0.00 seconds.


As you can see, the second call returns almost instantly, because the result was cached.

## Recursive function

In [6]:
@fleche
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

In [7]:
start = time.time()
fib(20)
end = time.time()
print(f'fib(20) took {end - start:.4f} seconds with caching.')

fib(20) took 0.0204 seconds with caching.


Without caching, this would be much slower as each call to `fib` would be recomputed.

## Caching Methods of User-defined Types

`fleche` can also cache methods of classes. For this to work, the class must be "digest-compatible". You can make a class digest-compatible by implementing a `__digest__` method or by using a `dataclass`.

In [8]:
class MyClass:
    def __init__(self, val):
        self.val = val
    
    def __digest__(self):
        # The digest defines how the instance is identified in the cache
        return Digest(str(self.val))

    @fleche
    def compute(self, x):
        print(f"Computing {self.val} + {x}...")
        time.sleep(1)
        return self.val + x

In [9]:
obj = MyClass(10)

start = time.time()
print(f"Result: {obj.compute(5)}")
print(f"First call took {time.time() - start:.2f} seconds.")

start = time.time()
print(f"Result: {obj.compute(5)}")
print(f"Second call (same instance) took {time.time() - start:.2f} seconds.")

Computing 10 + 5...


Result: 15
First call took 1.00 seconds.
Result: 15
Second call (same instance) took 0.00 seconds.


If you mutate the instance such that its digest changes, the cache will be missed.

In [10]:
obj.val = 20
start = time.time()
print(f"Result: {obj.compute(5)}")
print(f"Call after mutation took {time.time() - start:.2f} seconds.")

Computing 20 + 5...


Result: 25
Call after mutation took 1.00 seconds.


## Passing Digests as Arguments

`fleche` supports passing `Digest` objects directly to cached functions. When a function receives a `Digest`, `fleche` automatically expands it to its actual value from the cache before executing the function. You can use the convenience wrapper `D` to mark a string as a digest.

In [11]:
from fleche import D
from fleche.digest import digest as value_digest

@fleche
def double(x):
    print(f"Doubling {x}...")
    return x * 2

# 1. Run the calculation to ensure it is cached
long_running_calculation(10)

# 2. Compute the value digest for 100 (the cached result)
v = long_running_calculation(10)
val_dig = value_digest(v)
print(f"Value Digest: {val_dig}")

# 3. Pass a short digest prefix of the value to double(); it will expand to 100.
short = str(val_dig)[:8]
print(f"Short digest: {short}")
print(f"Result: {double(D(short))}")

Running calculation for 10...


Value Digest: 60079f7901a9295349d1796c037afc132e81286f785ddeeb763104ef02363102
Short digest: 60079f79
Doubling 100...
Result: 200


## Metadata

`fleche` allows you to add metadata to your cached functions using the `tags` context manager. This can be useful for organizing and querying your results.

In [12]:
@fleche
def another_calculation(a, b):
    return a + b

In [13]:
with tags(project='my_project', category='testing'):
    another_calculation(1, 2)
    another_calculation(3, 4)

This metadata is stored alongside the cached result. You can then use the `metadata.table` method to view the metadata for all cached results.

In [14]:
cache().table()

name  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...  long_running_calculation   
698e29f05ba00ee23503848cd166215f62cd976d5443159...                       fib   
405dfbadf453a9d5bbe3482fbb05251a6fc18b90045f8b4...                       fib   
24231d9bc47f7abc0ea485d178fc8457dce8082790f8c94...                       fib   
dabebddba19859bdb1a075e422b1842dde433c7d6ca51c2...                       fib   
e2b3a4eaf9036f83560677b9bad64cbb8263cf3496ba6fa...                       fib   
981a54293b3027931a47d21c275955d6dab2fe6d8fa4c2f...                       fib   
fdb0a9b8b1df91293924ad8fc03bcf041f3e50f924e9d19...                       fib   
db6b5632c77a01a692e13c486c54536a0c46907972d9390...                       fib   
cd94bda61e484ca82c470b6ab50c0f1b98055e6a905d82f...                       fib   
f4efb0e68400d195d8d57051088964d98b9ae4f7dbb78da...                       fib   
bc7f8b67d293d9fbdec343f49c44d46e655516537cc492b...                       fib   
6fbe26ceafe80276c7714b2aae14c25bd38e24683d6624f...                       fib   
f3cefd42fd07119792c38084ecbb33f1955ccd83d8021e1...                       fib   
2b7c9e8c5e0a73afddb28a49d47415462b3c84adc05c355...                       fib   
60bf26e7cdec62d1a8f9ebfbebb336850cf588d547badc7...                       fib   
9b23a905e639149d35169118ffefadec03a181cbd290006...                       fib   
18e77dab7a4a0385a66512547336e7a02ee62803634fd80...                       fib   
e1a78503325bcbb94116e03febed590b14bdd7f955c4f15...                       fib   
34814b3262b7920c7cef43eaf8a1e86e849611ed0d0317c...                       fib   
9214caae9e9819230882f7e9f0f1970fd5f2fc354c3404d...                       fib   
c5a5bb6cf5b69c7eb5e244ed540af44391dff12a77fcecf...                       fib   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...                   compute   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...                   compute   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...  long_running_calculation   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...                    double   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...       another_calculation   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...       another_calculation   

                                                                 arguments  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...                {'x': 2}   
698e29f05ba00ee23503848cd166215f62cd976d5443159...                {'n': 1}   
405dfbadf453a9d5bbe3482fbb05251a6fc18b90045f8b4...                {'n': 0}   
24231d9bc47f7abc0ea485d178fc8457dce8082790f8c94...                {'n': 2}   
dabebddba19859bdb1a075e422b1842dde433c7d6ca51c2...                {'n': 3}   
e2b3a4eaf9036f83560677b9bad64cbb8263cf3496ba6fa...                {'n': 4}   
981a54293b3027931a47d21c275955d6dab2fe6d8fa4c2f...                {'n': 5}   
fdb0a9b8b1df91293924ad8fc03bcf041f3e50f924e9d19...                {'n': 6}   
db6b5632c77a01a692e13c486c54536a0c46907972d9390...                {'n': 7}   
cd94bda61e484ca82c470b6ab50c0f1b98055e6a905d82f...                {'n': 8}   
f4efb0e68400d195d8d57051088964d98b9ae4f7dbb78da...                {'n': 9}   
bc7f8b67d293d9fbdec343f49c44d46e655516537cc492b...               {'n': 10}   
6fbe26ceafe80276c7714b2aae14c25bd38e24683d6624f...               {'n': 11}   
f3cefd42fd07119792c38084ecbb33f1955ccd83d8021e1...               {'n': 12}   
2b7c9e8c5e0a73afddb28a49d47415462b3c84adc05c355...               {'n': 13}   
60bf26e7cdec62d1a8f9ebfbebb336850cf588d547badc7...               {'n': 14}   
9b23a905e639149d35169118ffefadec03a181cbd290006...               {'n': 15}   
18e77dab7a4a0385a66512547336e7a02ee62803634fd80...               {'n': 16}   
e1a78503325bcbb94116e03febed590b14bdd7f955c4f15...               {'n': 17}   
34814b3262b7920c7cef43eaf8a1e86e849611ed0d0317c...               {'n': 18}   
9214caae9e9819230882f7e9f0f1970fd5f2fc354c3404d...               {'n': 19}   
c5a5bb6cf5b69c7eb5e244ed540af44391d

## Filtering

The metadata table is just pandas so you can query and filter as you like.

In [15]:
cache().table().query('name!="fib"')

name  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...  long_running_calculation   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...                   compute   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...                   compute   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...  long_running_calculation   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...                    double   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...       another_calculation   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...       another_calculation   

                                                                 arguments  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...                {'x': 2}   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...  {'self': '10', 'x': 5}   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...  {'self': '20', 'x': 5}   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...               {'x': 10}   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...              {'x': 100}   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...        {'a': 1, 'b': 2}   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...        {'a': 3, 'b': 4}   

                                                      module version  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...  __main__    None   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...  __main__    None   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...  __main__    None   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...  __main__    None   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...  __main__    None   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...  __main__    None   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...  __main__    None   

                                                   code_digest  result  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...        None       4   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...        None      15   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...        None      25   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...        None     100   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...        None     200   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...        None       3   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...        None       7   

                                                       timestart  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...  1.771979e+09   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...  1.771979e+09   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...  1.771979e+09   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...  1.771979e+09   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...  1.771979e+09   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...  1.771979e+09   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...  1.771979e+09   

                                                        timestop  walltime  \
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...  1.771979e+09  2.000305   
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...  1.771979e+09  1.000342   
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...  1.771979e+09  1.000332   
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...  1.771979e+09  2.000361   
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...  1.771979e+09  0.000080   
a8a3061653183cff08e5c414f9a4087550ab5e33cc186ae...  1.771979e+09  0.000019   
1e352b538b9219d4e5fcaf429882a4b3587f3bc6358ef0f...  1.771979e+09  0.000015   

                                                       project category  
4435e2927c194eff1bb90270ab3f353f0db1a8b50ec0493...         NaN      NaN  
81d4df70836ded7d8a0dd70348976c431210ff824d870ac...         NaN      NaN  
e9f9ca077d0566f335e54df480d202815d7b5b5ac5e7ccf...         NaN      NaN  
a7ce6824785adc406ca3561dcf98b3c64ddf1539d2467e1...         NaN      NaN  
3fb9bd0ba9c388530e4a0f5fbb789aaf7b5c598c1fae748...         NaN      NaN  
a8a3061653183cff08e5c4

## Querying Cached Calls via Function Wrapper

You can retrieve previously cached calls that match some of your function's arguments and metadata using the function wrapper's `query` method. Any field left as `None` is treated as a wildcard. Arguments and result are compared by digest internally, but the wrapper decodes them back to Python objects when returning matches.

Example using the `another_calculation` wrapper we created above:


In [16]:
# Query by metadata presence (tags) and a specific key-value filter
for call in another_calculation.query(1, 2, metadata={"tags": {}}):
    # presence-only: any call with 'tags'
    print(call.name, call.arguments, call.metadata.get("tags"))

for call in another_calculation.query(3, 4, metadata={"tags": {"project": "my_project"}}):
    # equality filter on metadata
    assert call.metadata["tags"]["project"] == "my_project"
    # arguments and result are decoded if they were stored as digests
    print(call.arguments, call.result)


another_calculation {'a': 1, 'b': 2} {'project': 'my_project', 'category': 'testing'}
{'a': 3, 'b': 4} 7


## Lazy Loading

When dealing with large results or many cached calls, you might not want to load everything at once. `fleche` supports "lazy loading", where arguments and results are only fetched from the cache when you actually access them.

In [17]:
from fleche.call import LazyCall

# 1. Get the digest for a known call
key = fib.digest(20)

# 2. Load it lazily
lazy_call = cache().load(key, lazy=True)
print(f"Obtained {type(lazy_call).__name__} for {lazy_call.name}(20)")

# 3. Accessing .result or .arguments will now trigger the load
print("Accessing result now (triggers load)...")
print(f"Result: {lazy_call.result}")

Obtained LazyCall for fib(20)
Accessing result now (triggers load)...
Result: 6765
